# Explainability for Classifiers: GradCAM, OcclusionSensitivity et al.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/mtwenzel/teaching/blob/master/01 Explainability using tf_explain.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/mtwenzel/teaching/blob/master/01 Explainability using tf_explain.ipynb
"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>
<br/><br/><br/>

This notebook shows for MNIST and for a medical example (Parkinson SPECT classification) how different visualization methods compare.

The code inherits from the 'tf_explain' original authors' example code and adapts it to the Parkinson example.

Use as a basis for own experiments.

# Preparations

In [0]:
#@title Install required python packages.
# https://github.com/sicara/tf-explain
try:
  import tf_explain as tfx
except:
  !pip install tf_explain
  import tf_explain as tfx

# https://github.com/CyberZHG/keras-radam
try:
  from keras_radam.training import RAdamOptimizer # for TF
except:
  !pip install keras-rectified-adam
  from keras_radam.training import RAdamOptimizer

In [0]:
#@title Install TensorFlow { display-mode: "form" }
TF_Installation = 'TF2 RC0 (GPU)' #@param ['TF2 Nightly (GPU)', 'TF2 RC0 (GPU)', 'TF2 Stable (GPU)', 'TF1 Nightly (GPU)', 'TF1 Stable (GPU)','System']
# added 2.0.0-rc0
if TF_Installation == 'TF2 Nightly (GPU)':
  !pip install -q --upgrade tf-nightly-gpu-2.0-preview
  print('Installation of `tf-nightly-gpu-2.0-preview` complete.')
elif TF_Installation == 'TF2 RC0 (GPU)':
  !pip install -q --upgrade tensorflow-gpu==2.0.0-rc0
  print('Installation of `tensorflow-gpu==2.0.0-rc0` complete. Use with tensorflow_probability=0.8.0-rc0')
elif TF_Installation == 'TF2 Stable (GPU)':
  !pip install -q --upgrade tensorflow-gpu==2.0.0-alpha0
  print('Installation of `tensorflow-gpu==2.0.0-alpha0` complete.')
elif TF_Installation == 'TF1 Nightly (GPU)':
  !pip install -q --upgrade tf-nightly-gpu
  print('Installation of `tf-nightly-gpu` complete.')
elif TF_Installation == 'TF1 Stable (GPU)':
  !pip install -q --upgrade tensorflow-gpu
  print('Installation of `tensorflow-gpu` complete.')
elif TF_Installation == 'System':
  pass
else:
  raise ValueError('Selection Error: Please select a valid '
                   'installation option.')

In [0]:
#@title Install tensorflow_probability { display-mode: "form" }
TFP_Installation = "0.8.0-rc0" #@param ["0.8.0-rc0", "Nightly", "Stable", "System"]

if TFP_Installation == "Nightly":
  !pip install -q tfp-nightly
  print("Installation of `tfp-nightly` complete.")
elif TFP_Installation == "0.8.0-rc0":
  !pip install -q --upgrade tensorflow-probability==0.8.0-rc0
  print("Installation of `tensorflow-probability` complete.")
elif TFP_Installation == "Stable":
  !pip install -q --upgrade tensorflow-probability
  print("Installation of `tensorflow-probability` complete.")
elif TFP_Installation == "System":
  pass
else:
  raise ValueError("Selection Error: Please select a valid "
                   "installation option.")

In [0]:
#@title Check GPU availability and TF version.
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))
print(tf.__version__ )# Has to be 2.0 for this notebook to work...

In [0]:
%load_ext tensorboard

In [0]:
#@title Imports/Definitions {display-mode:"form"}
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input,Conv2D,Dense,GlobalAveragePooling2D,concatenate,Flatten, MaxPooling2D, BatchNormalization, Dropout, SpatialDropout2D
from tensorflow.keras.applications import InceptionV3,DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd

import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

def get_data_generators(paths_dict, batch_size=32, target_size=(109,91)):
    '''
    Returns generators for batches of train/val/test data collected from the folders given to the function. 
    
    Params: 
        paths_dict: Dictionary with three paths in 'train', 'val', 'test' pointing to folders with images to use. Subfolders are assumed to contain classes. Mandatory.
        batch_size: Batch size. Default=32
        target_size: 2D size tuple to rescale the images to. Default=(109,91)
    Returns:
        train_generator, val_generator, test_generator: Generator functions to retrieve a batch of data
    '''
    train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, dtype=tf.float32)
    test_datagen =  ImageDataGenerator(rescale=1./255, dtype=tf.float32)
    train_generator = train_datagen.flow_from_directory(paths_dict['train'], 
                                                 target_size=target_size,
                                                 color_mode='grayscale',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

    val_generator = train_datagen.flow_from_directory(paths_dict['val'],
                                                 target_size=target_size,
                                                 color_mode='grayscale',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

    test_generator = test_datagen.flow_from_directory(paths_dict['test'], 
                                                 target_size=target_size,
                                                 color_mode='grayscale',
                                                 batch_size=566,
                                                 class_mode='categorical',
                                                 shuffle=False) # don't shuffle so that the file names and indices are in sync.
    

    return train_generator, val_generator, test_generator

In [0]:
#@title Prepare the data. {display-mode:'form'}
from urllib.request import urlopen
from zipfile import ZipFile

TARGET_SIZE = (96,96) # Square images because of visualization library...

zipurl = 'https://github.com/mtwenzel/parkinson-classification/raw/master/data/PPMI-classification.zip'
zipresp = urlopen(zipurl)
tempzip = open("PPMI-classification.zip", "wb")
tempzip.write(zipresp.read())
tempzip.close()
print("download complete, extracting...")

zf = ZipFile("PPMI-classification.zip")
zf.extractall(path = 'data/')
zf.close()
print("... done")

paths_dict = {'train': '/content/data/PPMI-classification/all_2d_train',
             'val': '/content/data/PPMI-classification/all_2d_val',
             'test': '/content/data/PPMI-classification/all_2d_val'}

train_generator, val_generator, test_generator = get_data_generators(paths_dict, target_size=TARGET_SIZE)

# Model Definition and Training

In [0]:
#@title Model definition
input_image = Input(shape=TARGET_SIZE+(1,))

x = BatchNormalization()(input_image)
x = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3,3), activation='relu', strides=(2,2), name='EarlyConv')(x)

x = BatchNormalization()(x)
x = Conv2D(filters=96, kernel_size=(3,3), activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=96, kernel_size=(3,3), activation='relu', strides=(2,2))(x)

x = BatchNormalization()(x)
x = Conv2D(filters=96, kernel_size=(3,3), activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=96, kernel_size=(3,3), activation='relu', strides=(2,2))(x)

x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3,3), activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', name='LastConv')(x)

x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(rate=0.25)(x)

preds = Dense(2,activation='softmax')(x) #final layer with softmax activation

model = Model(inputs=input_image,outputs=preds)

In [0]:
#@title Simple model definition
input_image = Input(shape=TARGET_SIZE+(1,))

x = Conv2D(filters=16, kernel_size=(7,7), activation='relu', name='EarlyConv')(input_image)
x = Conv2D(filters=32, kernel_size=(5,5), activation='relu')(x)
x = Conv2D(filters=64, kernel_size=(5,5), activation='relu')(x)
x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', name='LastConv')(x)

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(rate=0.25)(x)

preds = Dense(2,activation='softmax')(x) #final layer with softmax activation

model = Model(inputs=input_image,outputs=preds)
model.summary()

In [0]:
radam = RAdamOptimizer(learning_rate=1e-3)
model.compile(optimizer=radam, loss='categorical_crossentropy', metrics = ['accuracy'])

In [0]:
#@title Create the callbacks for visualization
#@markdown To provide some illustration, several callbacks are instantiated. Not all are used below, though.

#@markdown Double-click the header row to expand this cell and inspect the definitions. 

x_val_g = val_generator.next()
x_val_img = np.array(x_val_g[0])
x_val_lbl = np.array(x_val_g[1])
val_class_zero = (np.array([
    el for el, label in zip(x_val_img, x_val_lbl)
    if np.all(label == np.array([1] + [0]))
][0:9]), None)
val_class_one = (np.array([
    el for el, label in zip(x_val_img, x_val_lbl)
    if np.all(label == np.array([0] + [1]))
][0:9]), None)

cam_cb_00 = tfx.callbacks.GradCAMCallback(val_class_zero, layer_name='LastConv', class_index=0)
cam_cb_01 = tfx.callbacks.GradCAMCallback(val_class_zero, layer_name='LastConv', class_index=1)
cam_cb_10 = tfx.callbacks.GradCAMCallback(val_class_one, layer_name='LastConv', class_index=0)
cam_cb_11 = tfx.callbacks.GradCAMCallback(val_class_one, layer_name='LastConv', class_index=1)
occ_cb_00 =  tfx.callbacks.OcclusionSensitivityCallback(val_class_zero,class_index=0, patch_size=8)
occ_cb_01 =  tfx.callbacks.OcclusionSensitivityCallback(val_class_zero,class_index=1, patch_size=8)
occ_cb_10 =  tfx.callbacks.OcclusionSensitivityCallback(val_class_one,class_index=0, patch_size=8)
occ_cb_11 =  tfx.callbacks.OcclusionSensitivityCallback(val_class_one,class_index=1, patch_size=8)
tf_cb = tf.keras.callbacks.TensorBoard(histogram_freq=5)

In [0]:
# train the model on the new data for a few epochs. Use the callbacks only afterwards to speed up the process.
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=train_generator.n//train_generator.batch_size,
                              epochs=50,
                             validation_data=val_generator,
                             validation_steps=val_generator.n//val_generator.batch_size,
                             verbose=1,
                             callbacks=[tf_cb])

In [0]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.legend()
plt.show()
plt.plot(history.history['accuracy'], label='acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.title('Accuracy')
plt.legend()
plt.show()

In [0]:
# After that, only train two epochs to generate the visualizations. This is costly!
# Look into the embedded TensorBoard above to see results.
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=train_generator.n//train_generator.batch_size,
                              epochs=30,
                             validation_data=val_generator,
                             validation_steps=val_generator.n//val_generator.batch_size,
                             verbose=2,
                             callbacks=[cam_cb_00, cam_cb_01])

In [0]:
import datetime, os

logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

# Proceed to prediction

In [0]:
# Define a prediction function from the model, setting the learning phase to "learn" to let dropout be active.
# Caveat: Don't use with e.g. BatchNorm.
f = K.function([model.layers[0].input, K.learning_phase()],
               [model.layers[-1].output])

# This takes some memory!
def predict_with_uncertainty(f, x, no_classes, n_iter=20):
    result = np.zeros((n_iter,) + (x.shape[0], no_classes) )

    for i in range(n_iter):
        result[i,:, :] = f((x, 1))[0]
    prediction = result.mean(axis=0)
    uncertainty = result.std(axis=0)
    return prediction, uncertainty, result

In [0]:
%%time 
x_test = test_generator.next()
x_test = np.array(x_test[0])
p,u,r = predict_with_uncertainty(f,x_test[0:10],2)

In [0]:
plot, axs = plt.subplots(nrows=2, ncols=5, figsize=(16,6))
sns.distplot(r[:,0,0], bins=20, ax=axs[0][0])
sns.distplot(r[:,1,0], bins=20, ax=axs[1][0])
sns.distplot(r[:,2,0], bins=20, ax=axs[0][1])
sns.distplot(r[:,3,0], bins=20, ax=axs[1][1])
sns.distplot(r[:,4,0], bins=20, ax=axs[0][2])
sns.distplot(r[:,5,0], bins=20, ax=axs[1][2])
sns.distplot(r[:,6,0], bins=20, ax=axs[0][3])
sns.distplot(r[:,7,0], bins=20, ax=axs[1][3])
sns.distplot(r[:,8,0], bins=20, ax=axs[0][4])
sns.distplot(r[:,9,0], bins=20, ax=axs[1][4])

In [0]:
T = 10
Yt_hat = np.array([model.predict_generator(test_generator, steps=1) for _ in range(T)])

In [0]:
model_metrics = model.evaluate_generator(test_generator, steps=1)
predictions = model.predict_generator(test_generator, steps=1)
preds = list(zip(predictions[:,0],predictions[:,1]))

print(model.metrics_names, model_metrics)

result = list(zip(test_generator.filenames, preds))
with open('results_BaselineModel2D_ppmiOnUKEall_Uncertainty', 'w') as f:
    f.write(repr(result))

# A toy example from the `tf_explain` authors.

In [0]:
import numpy as np
import tensorflow as tf
import tf_explain

INPUT_SHAPE = (28, 28, 1)
NUM_CLASSES = 10

AVAILABLE_DATASETS = {
    'mnist': tf.keras.datasets.mnist,
    'fashion_mnist': tf.keras.datasets.fashion_mnist,
}
DATASET_NAME = 'mnist'  # Choose between "mnist" and "fashion_mnist"

# Load dataset
dataset = AVAILABLE_DATASETS[DATASET_NAME]
(train_images, train_labels), (test_images, test_labels) = dataset.load_data()

# Convert from (28, 28) images to (28, 28, 1)
train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]

# One hot encore labels 0, 1, .., 9 to [0, 0, .., 1, 0, 0]
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=NUM_CLASSES)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=NUM_CLASSES)

# Create model
img_input = tf.keras.Input(INPUT_SHAPE)

x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(img_input)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', name='target_layer')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.Model(img_input, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Select a subset of the validation data to examine
# Here, we choose 5 elements with label "0" == [1, 0, 0, .., 0]
validation_class_zero = (np.array([
    el for el, label in zip(test_images, test_labels)
    if np.all(label == np.array([1] + [0] * 9))
][0:5]), None)
# Select a subset of the validation data to examine
# Here, we choose 5 elements with label "4" == [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
validation_class_fours = (np.array([
    el for el, label in zip(test_images, test_labels)
    if np.all(label == np.array([0] * 4 + [1] + [0] * 5))
][0:5]), None)

# Instantiate callbacks
# class_index value should match the validation_data selected above
callbacks = [
    tf_explain.callbacks.GradCAMCallback(validation_class_zero, 'target_layer', class_index=0),
    tf_explain.callbacks.GradCAMCallback(validation_class_fours, 'target_layer', class_index=4),
    tf_explain.callbacks.ActivationsVisualizationCallback(validation_class_zero, layers_name=['target_layer']),
    tf_explain.callbacks.SmoothGradCallback(validation_class_zero, class_index=0, num_samples=15, noise=1.),
    tf_explain.callbacks.OcclusionSensitivityCallback(validation_class_zero, class_index=0, patch_size=4),
]

# Start training
model.fit(train_images, train_labels, epochs=5, callbacks=callbacks)

11493376/11490434 [==============================] - 0s 0us/step
Train on 60000 samples
Epoch 1/5
59840/60000 [============================>.] - ETA: 0s - loss: 0.4238 - accuracy: 0.8994

TypeError: ignored